In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV, ElasticNet, ElasticNetCV, SGDRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
import tensorflow as tf
import keras
import keras_tuner as kt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input, BatchNormalization
from tensorflow.keras.regularizers import l2
# Evaluation (Metrics & DM-Test)
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from itertools import islice
from math import lgamma, fabs, isnan, nan, exp, log, log1p, sqrt
from typing import Sequence, Callable, List, Tuple
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA

#### **1. Data Preparation**

In [2]:
df = pd.read_csv("df_ff_factors_100325.csv")
df = df.drop(columns=["Unnamed: 0","crsp_portno"])
df = df.sort_values(by='date')
df

df_tech = pd.read_csv("df_ff_factors_techfunds.csv")
df_tech = df_tech.drop(columns=["Unnamed: 0","crsp_portno"])
df_tech = df_tech.sort_values(by='date')

The chunk below describes the various functions involved in the training of the models, which are described in greater depth below:

(1) Generation of lagged dataframe i.e. lagging the characteristics

(2) Generation of stepped dataframe i.e. lagged characteristics being together with the 1-month ahead forecast

(3) Demeaning function for fund-level characteristics, which is important to ensure accuracy in the overall forecasting of annualised alphas

In [3]:
# Generate Lagged Dataset
def create_lagged_dataset(dataset, lag, target_var, id):
    lagged_dataset = dataset.copy()
    columns_list = list(lagged_dataset.columns)
    data_join = {}
    for column in columns_list:
        if column == target_var:
            data_join[column] = lagged_dataset[column]
        for n in range(1,lag+1):
            data_join[F'{column}_L{n}'] = lagged_dataset.groupby(id)[column].shift(n)
    lagged_dataset = pd.concat(data_join.values(), axis=1, ignore_index = True)
    lagged_dataset.columns = data_join.keys()
    return lagged_dataset.dropna()

# Generate Stepped Dataset for Training
## Steps is the number of months ahead that we are forecasting, e.g. step=2 is 2 months ahead.
## Note step=1 results in no change to dataset, i.e. use generated lagged variables to forecast current. 
def create_stepped_dataset(dataset, step, target_var, id):
    
    shifted_dataset = dataset.copy()
    shifted_dataset['shifted_target'] = shifted_dataset.groupby(id)[target_var].shift(-step + 1)
    
    # Drop rows where the shifted target is NaN (these occur due to the shift operation)
    shifted_dataset = shifted_dataset.dropna(subset=['shifted_target'])
    
    # Separate the features (X) and the target (y)
    X = shifted_dataset.drop(columns=[target_var, 'shifted_target'])
    y = shifted_dataset[['shifted_target']]
    y = y.rename(columns={'shifted_target':target_var})
    return X, y

def demeaning_fund_char(dataset, id, characteristic):
    for col in characteristic:
        dataset[f'demeaned_{col}'] = dataset[col] - dataset.groupby(id)[col].transform('mean')
    return dataset

In [4]:
import pandas as pd

# Assuming 'dataset' is your DataFrame and 'normalized_flow' is the column with NaN values
df['normalised_flow'] = df['normalised_flow'].fillna(method='ffill')
char_to_demean = ["exp_ratio", "turn_ratio", "normalised_flow", "shortrun_momentum"]
df_demeaned = demeaning_fund_char(df, id="crsp_fundno", characteristic = char_to_demean)
df_demeaned = df_demeaned.drop(columns = char_to_demean)   # drop original columns
df.head()

df_tech['normalised_flow'] = df_tech['normalised_flow'].fillna(method='ffill')
char_to_demean = ["exp_ratio", "turn_ratio", "normalised_flow", "shortrun_momentum"]
df_tech_demeaned = demeaning_fund_char(df_tech, id="crsp_fundno", characteristic = char_to_demean)
df_tech_demeaned = df_tech_demeaned.drop(columns = char_to_demean)   # drop original columns

df_tech

C:\Users\wjlwi\AppData\Local\Temp\ipykernel_20108\1523674970.py:4: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['normalised_flow'] = df['normalised_flow'].fillna(method='ffill')
C:\Users\wjlwi\AppData\Local\Temp\ipykernel_20108\1523674970.py:10: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_tech['normalised_flow'] = df_tech['normalised_flow'].fillna(method='ffill')


,crsp_fundno,date,mth_return,exp_ratio,turn_ratio,normalised_flow,gdp_to_debt_ratio,gdp_growth_rate,unm_rate,infl_rate,...,rolling_sharpe,mkt_return,rolling_alpha_3f,rolling_alpha_4f,rolling_alpha_5f,shortrun_momentum,demeaned_exp_ratio,demeaned_turn_ratio,demeaned_normalised_flow,demeaned_shortrun_momentum
43631,32553.0,1993-08-31,0.068935,0.0162,0.15,0.009529,64.101,3.5225,6.8,2.90,...,3.248937,0.056070,0.068910,0.068910,0.068910,0.012866,0.000025,-0.464384,0.001507,0.020627
43632,32553.0,1993-09-30,-0.004082,0.0162,0.15,0.014881,64.101,3.5225,6.7,2.90,...,2.216452,0.027009,-0.004108,-0.004108,-0.004108,-0.009113,0.000025,-0.464384,0.006859,-0.001352
43633,32553.0,1993-10-29,0.040984,0.0162,0.15,0.030641,64.669,3.5225,6.8,2.90,...,3.058133,0.021609,0.040962,0.040962,0.040962,0.000383,0.000025,-0.464384,0.022620,0.008144
13747,12051.0,1993-10-29,0.034712,0.0188,0.77,0.070030,64.669,3.5225,6.8,2.90,...,5.744340,0.021609,0.034690,0.034690,0.034690,0.013103,0.007662,-0.699757,0.066601,0.016976
13748,12051.0,1993-11-30,-0.075974,0.0188,0.77,0.000259,64.669,3.5225,6.6,2.90,...,-0.536935,-0.010806,-0.075999,-0.075999,-0.075999,-0.026033,0.007662,-0.699757,-0.003170,-0.022160
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3031,4610.0,2024-06-28,0.078565,0.0217,0.23,-0.650182,120.040,2.5427,4.1,3.35,...,1.619481,0.055058,0.001428,0.002329,0.003201,-0.025985,-0.001211,-1.434538,-0.575797,-0.025378
1960,4330.0,2024-07-31,0.028831,0.0077,0.31,-0.022800,120.731,2.5427,4.2,3.35,...,0.691406,0.055058,0.004765,0.005643,0.010978,-0.024241,-0.000815,-0.039661,-0.033919,-0.013655
2113,4333.0,2024-07-31,0.028826,0.0076,0.31,-0.402964,120.731,2.5427,4.2,3.35,...,0.691738,0.055058,0.004733,0.005601,0.010953,-0.024228,-0.000737,-0.039661,-0.817815,-0.013661
1842,4327.0,2024-07-31,0.028616,0.0102,0.31,-0.032107,120.731,2.5427,4.2,3.35,...,0.676394,0.055058,0.004506,0.005384,0.010697,-0.024451,-0.000815,-0.039661,-0.042810,-0.013657


In [5]:
# Generates next date
def generate_next_date(list_of_dates, date):
    return list_of_dates[list_of_dates > date].min()

def process_factor_model(X_factor, y_factor, train_end, test_date):
    X_train = X_factor[X_factor['date_L1'] <= train_end].drop(columns='date_L1')
    X_test = X_factor[X_factor['date_L1'] == test_date].drop(columns='date_L1')

    y_train = y_factor.loc[X_train.index]
    y_test = y_factor.loc[X_test.index]
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    return X_train_scaled, X_test_scaled, y_train, y_test

#### **2. Model Training (Alpha)**

Asset-Pricing Factor Models:
* 5-Factor + MOM Model: mktrf, SMB, HML, RMW, CMA, UMD

The overall_function trains the models using the final selected parameters after hyperparameter tuning.

Sample code for Regime-Switching Model

Overall Training Cycle Code

In [ ]:
# OLD HYPERPARAMS
# lstm_hyperparams_dict_healthcare = dict(
#     {'2019': dict({
#         'num_layers':3,
#         'units':[96,32,32],
#         'optimizer':'Adam',
#         'drop_out':[0.2,0.2,0.1],
#         'activation':['tanh','tanh','linear'],
#         'lr':0.019962442034576384
#     }),
#      '2020': dict({
#         'num_layers':3,
#         'units':[64,32,32],
#         'optimizer':'Nadam',
#         'drop_out':[0.1,0.1,0.1],
#         'activation':['tanh','tanh','linear'],
#         'lr':0.021504061608420576
#     }),
#      '2021': dict({
#         'num_layers':3,
#         'units':[96,32,32],
#         'optimizer':'Nadam',
#         'drop_out':[0.1,0.1,0.1],
#         'activation':['tanh','tanh','linear'],
#         'lr':0.012302249555768368
#     }),
#      '2022': dict({
#         'num_layers':3,
#         'units':[96,32,32],
#         'optimizer':'Nadam',
#         'drop_out':[0.1,0.2,0.2],
#         'activation':['tanh','tanh','linear'],
#         'lr':0.010092957472413086
#     }),
#      '2023': dict({
#         'num_layers':2,
#         'units':[64,32],
#         'optimizer':'Adam',
#         'drop_out':[0.2,0.2],
#         'activation':['tanh','tanh'],
#         'lr':0.018388080425636798
#     }),
#      '2024': dict({
#         'num_layers':2,
#         'units':[32,32],
#         'optimizer':'Adam',
#         'drop_out':[0.1,0.2],
#         'activation':['tanh','tanh'],
#         'lr':0.007947588699235507
#     })}
# )

rf_hyperparams_dict_healthcare = dict(
    {'2019': dict({
        'n_estimators':300,
        'min_samples_split':5,
        'min_samples_leaf':4,
        'max_features':'log2',
        'max_depth':20
    }),
     '2020': dict({
        'n_estimators':100,
        'min_samples_split':10,
        'min_samples_leaf':4,
        'max_features':'log2',
        'max_depth':None
    }),
     '2021': dict({
        'n_estimators':300,
        'min_samples_split':5,
        'min_samples_leaf':4,
        'max_features':'log2',
        'max_depth':20
    }),
     '2022': dict({
        'n_estimators':100,
        'min_samples_split':10,
        'min_samples_leaf':4,
        'max_features':'log2',
        'max_depth':None
    }),
     '2023': dict({
        'n_estimators':300,
        'min_samples_split':5,
        'min_samples_leaf':4,
        'max_features':'log2',
        'max_depth':20
    }),
     '2024': dict({
        'n_estimators':100,
        'min_samples_split':10,
        'min_samples_leaf':4,
        'max_features':'log2',
        'max_depth':None
    })}
)

Healthcare Mutual Funds

In [6]:
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV
from lightgbm import LGBMRegressor

# Define a simplified XGBoost parameter grid
xgb_param_grid = {
    'n_estimators': [100, 300, 500],  # Number of boosting rounds
    'learning_rate': [0.01, 0.05, 0.1],  # Learning rate (step size)
    'max_depth': [3, 5, 7],  # Maximum depth of a tree
    'subsample': [0.7, 0.8, 0.9],  # Fraction of samples for training each tree
    'colsample_bytree': [0.7, 0.8, 1.0]  # Fraction of features for each tree
}

def xgb_tuner(X_train, y_train, xgb_param_grid):
    # Initialize XGBRegressor
    xgb_model = XGBRegressor(random_state=42, n_jobs=-1)

    # Use RandomizedSearchCV for efficiency
    xgb_random_search = RandomizedSearchCV(
        estimator=xgb_model,
        param_distributions=xgb_param_grid,
        n_iter=10,  # Number of parameter settings tested
        cv=TimeSeriesSplit(n_splits=5),  # TimeSeriesSplit for time-dependent data
        scoring='neg_mean_squared_error',  # Evaluate using negative mean squared error
        verbose=2,
        random_state=42,
        n_jobs=-1
    )

    # Perform hyperparameter tuning
    xgb_random_search.fit(X_train, y_train.values.ravel())

    # Best hyperparameters
    best_xgb_params = xgb_random_search.best_params_
    print(f'Best XGBoost Parameters: {best_xgb_params}')
    return best_xgb_params

# Define a simplified LGBM parameter grid
lgbm_param_grid = {
    'n_estimators': [100, 300, 500],  # Number of boosting rounds
    'learning_rate': [0.01, 0.05, 0.1],  # Learning rate (step size)
    'max_depth': [-1, 3, 5],  # Maximum depth of a tree, -1 means no limit
    'num_leaves': [31, 63, 127],  # Number of leaves in a tree (controls complexity)
    'subsample': [0.7, 0.8, 0.9],  # Fraction of samples used for training each tree
    'colsample_bytree': [0.7, 0.8, 1.0]  # Fraction of features used for each tree
}

def lgbm_tuner(X_train, y_train, lgbm_param_grid):
    # Initialize LGBMRegressor
    lgbm_model = LGBMRegressor(random_state=42, n_jobs=-1)

    # Use RandomizedSearchCV for efficiency
    lgbm_random_search = RandomizedSearchCV(
        estimator=lgbm_model,
        param_distributions=lgbm_param_grid,
        n_iter=10,  # Number of parameter settings tested
        cv=TimeSeriesSplit(n_splits=5),  # TimeSeriesSplit for time-dependent data
        scoring='neg_mean_squared_error',  # Evaluate using negative mean squared error
        verbose=2,
        random_state=42,
        n_jobs=-1
    )

    # Perform hyperparameter tuning
    lgbm_random_search.fit(X_train, y_train.values.ravel())

    # Best hyperparameters
    best_lgbm_params = lgbm_random_search.best_params_
    print(f'Best LGBM Parameters: {best_lgbm_params}')
    return best_lgbm_params

In [7]:
# Training Cycle
from tensorflow.keras.optimizers import Adam, Nadam, Adagrad
from tensorflow.keras.models import Sequential, load_model
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import TimeSeriesSplit
def overall_function(dataset, outcome, 
                     lstm_hyperparams_dict_healthcare, rf_hyperparams_dict_healthcare):
    dataset['date'] = pd.to_datetime(dataset['date']) # converting to date format
    dataset = dataset.sort_values(by='date')
    df_factor = dataset.drop(columns=['mkt_return','mth_return','rf']) # remove irrelevant variables

    # Creating Lagged and Stepped Datasets
    X_dataset, y_dataset = create_stepped_dataset(create_lagged_dataset(df_factor, lag=1,target_var=outcome, id = 'crsp_fundno'),step=1,target_var=outcome, id = 'crsp_fundno_L1')
    # return(X_dataset)
    X_dataset = X_dataset.drop(columns=['crsp_fundno_L1'], errors='ignore')

    list_of_dates = pd.to_datetime(X_dataset['date_L1'])
    percentile_70 = list_of_dates.quantile(0.7) # 70-30 split
    train_end = list_of_dates.loc[(list_of_dates - percentile_70).abs().idxmin()]
    df_end = list_of_dates.max()
    # return(y_dataset)
    results = []

    ## Implement cross-validation split
    tscv = TimeSeriesSplit(n_splits = 5)
    
    while train_end != df_end:
        
        test_date = generate_next_date(list_of_dates, train_end)
        if pd.isna(test_date):
            break 

        # Process data for modeling
        X_train, X_test, y_train, y_test = process_factor_model(X_dataset, y_dataset, train_end, test_date)
        
        # For Adding Results
        df_in_loop = y_test.copy()
        
        ### Model 1: Lasso Regression
        lasso_cv = LassoCV(cv = tscv, random_state = 18, max_iter = 100000)
        lasso_cv.fit(X_train, y_train)
        
        # Create the Lasso model with the optimal alpha value
        lasso_model = Lasso(alpha = lasso_cv.alpha_)
        lasso_model.fit(X_train, y_train)
        lassopred = lasso_model.predict(X_test)
        # Adding Linear Model
        df_in_loop[f'lasso'] = lassopred
        print('Lasso Done')
        
        ### Model 1: Ridge Regression ###
        ridge_cv = RidgeCV(cv = tscv)
        ridge_cv.fit(X_train, y_train)
    
        ridge_model = Ridge(alpha = ridge_cv.alpha_)
        ridge_model.fit(X_train, y_train)
        
        ridgepred = ridge_model.predict(X_test)
        # Adding Linear Model
        df_in_loop[f'ridge'] = ridgepred
        print('Ridge Done')
        
        ### Model 2: LSTM ###
        # X_train_lstm = np.array(X_train).astype(np.float32)
        # X_test_lstm = np.array(X_test).astype(np.float32)
        # y_train_lstm = np.array(y_train).astype(np.float32)
        
        test_year = test_date.year
        # print(f'test_year is {test_year} with hyperparameters of {lstm_hyperparams_dict_healthcare[str(test_year)]}')
        # lstm_model = load_model(f'best_lstm_model_{train_end.year}.keras')
        
        # num_layers = len(lstm_hyperparams_dict_healthcare[str(test_year)]['units'])
        
        # # Building LSTM Model
        # lstm_model = Sequential()
        # for layer_num in range(num_layers):
        #     is_last_layer = (layer_num == (num_layers - 1))  # Check if it's the last layer
        #     lstm_model.add(LSTM(
        #         units=lstm_hyperparams_dict_healthcare[str(test_year)]['units'][layer_num], 
        #         return_sequences=not is_last_layer,  # Only last layer has return_sequences=False
        #         input_shape=(X_train_lstm.shape[1], 1) if layer_num == 0 else None,  # Define input shape only for the first layer
        #         activation=lstm_hyperparams_dict_healthcare[str(test_year)]['activation'][layer_num]
        #     ))
        #     lstm_model.add(Dropout(lstm_hyperparams_dict_healthcare[str(test_year)]['drop_out'][layer_num]))
        # # Output Layer
        # lstm_model.add(Dense(units=1))
        
        # # Compilation
        # if lstm_hyperparams_dict_healthcare[str(test_year)]['optimizer'] == 'Adam':
        #     lstm_model.compile(
        #         optimizer=Adam(learning_rate=lstm_hyperparams_dict_healthcare[str(test_year)]['lr']),
        #         loss='mean_squared_error',
        #         metrics=[
        #             tf.keras.metrics.RootMeanSquaredError()
        #         ]
        #     )
        # elif lstm_hyperparams_dict_healthcare[str(test_year)]['optimizer'] == 'Nadam':
        #     lstm_model.compile(
        #         optimizer=Nadam(learning_rate=lstm_hyperparams_dict_healthcare[str(test_year)]['lr']),
        #         loss='mean_squared_error',
        #         metrics=[
        #             tf.keras.metrics.RootMeanSquaredError()
        #         ]
        #     )

        # Early stopping callback
        # callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

        # # Train the model
        # lstm_model.fit(
        #     X_train_lstm, y_train_lstm,
        #     epochs=10, batch_size=512,
        #     callbacks=[callback]
        # )

        # lstmpred = lstm_model.predict(X_test_lstm)
        # df_in_loop[f'lstm'] = lstmpred
        # print('LSTM Done')
        
        optimal_param_xgb = xgb_tuner(X_train, y_train, xgb_param_grid)
        xgboost_model = xgb.XGBRegressor(subsample=optimal_param_xgb['subsample'],
                                         n_estimators=optimal_param_xgb['n_estimators'],
                                         max_depth=optimal_param_xgb['max_depth'],
                                         learning_rate=optimal_param_xgb['learning_rate'],
                                         colsample_bytree=optimal_param_xgb['colsample_bytree'],
                                         random_state=18)
        xgboost_model.fit(X_train, y_train)
        df_in_loop[f'xgboost'] = xgboost_model.predict(X_test)
        print('XGBoost done')
        #errors(xgboost_model, 'gradientboost', X_train, y_train, errors_path, window_end+1, pred_train_path)

        ## 16. LightGBM
        optimal_param_lgbm = lgbm_tuner(X_train, y_train, lgbm_param_grid)
        lightgbm_model = lgb.LGBMRegressor(objective='regression', 
                                           subsample = optimal_param_lgbm['subsample'],
                                           num_leaves = optimal_param_lgbm['num_leaves'],
                                           n_estimators = optimal_param_lgbm['n_estimators'],
                                           max_depth= optimal_param_lgbm['max_depth'],
                                           learning_rate=optimal_param_lgbm['learning_rate'],
                                           colsample_bytree=optimal_param_lgbm['colsample_bytree'],
                                           random_state=18, verbosity=-1)
        lightgbm_model.fit(X_train, y_train)
        df_in_loop[f'lgbm'] = lightgbm_model.predict(X_test)
        print('LGBM done')
                
        ### Model 3: Random Forest Regression ###
        print(f'test_year is {test_year} with hyperparameters of {rf_hyperparams_dict_healthcare[str(test_year)]}')
        rf_model = RandomForestRegressor(n_estimators=rf_hyperparams_dict_healthcare[str(test_year)]['n_estimators'], 
                                         min_samples_split=rf_hyperparams_dict_healthcare[str(test_year)]['min_samples_split'],
                                         min_samples_leaf=rf_hyperparams_dict_healthcare[str(test_year)]['min_samples_leaf'],
                                         max_features = rf_hyperparams_dict_healthcare[str(test_year)]['max_features'], 
                                         max_depth=rf_hyperparams_dict_healthcare[str(test_year)]['max_depth'],
                                         random_state=40, n_jobs=-1)

        rf_model.fit(X_train, y_train.values.ravel())
        rf_pred = rf_model.predict(X_test)
        # return(y_pred)
        df_in_loop[f'rf'] = rf_pred
        
        print('RF Done')
        
        ### Model 4: PCA ###
        X_train = pd.DataFrame(X_train)
        X_test = pd.DataFrame(X_test)

        X_train_pure = X_train
        X_test_pure = X_test
        
        pca = PCA()
        pca.fit(X_train_pure)
        cumulative_variance_ratio = np.cumsum(pca.explained_variance_ratio_)
          #to explain more than 85% of the variance
        num_components = np.where(cumulative_variance_ratio >= 0.85)[0][0] + 1 
        pca_new = PCA(n_components=num_components)
        X_train_pca = pca_new.fit_transform(X_train_pure)
        X_train_pca = pd.DataFrame(X_train_pca)
        X_train_pca.columns = X_train_pca.columns.astype(str)

        X_test_pca = pca_new.transform(X_test_pure)
        X_test_pca = pd.DataFrame(X_test_pca)
        X_test_pca.columns = X_test_pca.columns.astype(str)
        
        pure_factor_model = LinearRegression()
        pure_factor_model.fit(X_train_pca, y_train)
        y_pred = pure_factor_model.predict(X_test_pca)
        df_in_loop[f'pca'] = y_pred
        print('PCA Done')
        
        # return(y_pred)
        # Add results into loop
        results.append(df_in_loop)
        train_end = test_date
        num_remaining_dates = len(list(set(date for date in list_of_dates if date > test_date)))
        print(f'{num_remaining_dates} dates remaining')
    # return(results)
    combined_df = pd.concat(results, ignore_index=True)
    return combined_df


In [ ]:
# Run to train the models
y_new = overall_function(dataset=df, outcome = "rolling_alpha_5f", 
                         lstm_hyperparams_dict_healthcare=None, rf_hyperparams_dict_healthcare=rf_hyperparams_dict_healthcare)

C:\Users\wjlwi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\linear_model\_coordinate_descent.py:1656: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Lasso Done
Ridge Done
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best XGBoost Parameters: {'subsample': 0.7, 'n_estimators': 500, 'max_depth': 3, 'learning_rate': 0.01, 'colsample_bytree': 0.7}
XGBoost done
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best LGBM Parameters: {'subsample': 0.8, 'num_leaves': 63, 'n_estimators': 100, 'max_depth': 3, 'learning_rate': 0.05, 'colsample_bytree': 0.8}


C:\Users\wjlwi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LGBM done
test_year is 2019 with hyperparameters of {'n_estimators': 300, 'min_samples_split': 5, 'min_samples_leaf': 4, 'max_features': 'log2', 'max_depth': 20}
RF Done
PCA Done
57 dates remaining


C:\Users\wjlwi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\linear_model\_coordinate_descent.py:1656: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


...

LGBM done
test_year is 2024 with hyperparameters of {'n_estimators': 100, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_features': 'log2', 'max_depth': None}
RF Done
PCA Done
1 dates remaining


C:\Users\wjlwi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\linear_model\_coordinate_descent.py:1656: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Lasso Done
Ridge Done
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best XGBoost Parameters: {'subsample': 0.8, 'n_estimators': 300, 'max_depth': 5, 'learning_rate': 0.01, 'colsample_bytree': 1.0}
XGBoost done
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best LGBM Parameters: {'subsample': 0.8, 'num_leaves': 63, 'n_estimators': 100, 'max_depth': 3, 'learning_rate': 0.05, 'colsample_bytree': 0.8}


C:\Users\wjlwi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LGBM done
test_year is 2024 with hyperparameters of {'n_estimators': 100, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_features': 'log2', 'max_depth': None}
RF Done
PCA Done
0 dates remaining


In [21]:
y_new.to_csv('tuned_results_xgboostlgbm_yearly_tuned_every_round.csv')

Technology Sector

In [9]:
# lstm_hyperparams_dict_tech = dict(
#     {'2019': dict({
#         'num_layers':2,
#         'units':[32,32],
#         'optimizer':'Adam',
#         'drop_out':[0.1,0.1],
#         'activation':['linear','tanh'],
#         'lr':0.04130929884537057
#     }),
#      '2020': dict({
#         'num_layers':2,
#         'units':[96,32],
#         'optimizer':'Adam',
#         'drop_out':[0.1,0.1],
#         'activation':['tanh','tanh'],
#         'lr':0.0017545326289340611
#     }),
#      '2021': dict({
#         'num_layers':2,
#         'units':[96,32],
#         'optimizer':'Adam',
#         'drop_out':[0.1,0.2],
#         'activation':['linear','tanh'],
#         'lr':0.008514542675036612
#     }),
#      '2022': dict({
#         'num_layers':2,
#         'units':[32,32],
#         'optimizer':'Adagrad',
#         'drop_out':[0.2,0.1],
#         'activation':['linear','tanh'],
#         'lr':0.06210952184736157
#     }),
#      '2023': dict({
#         'num_layers':4,
#         'units':[128,32,32,32],
#         'optimizer':'Adam',
#         'drop_out':[0.2,0.1,0.1,0.2],
#         'activation':['tanh','tanh','linear','linear'],
#         'lr':0.019671144721368838
#     }),
#      '2024': dict({
#         'num_layers':2,
#         'units':[32,32],
#         'optimizer':'Adam',
#         'drop_out':[0.2,0.1],
#         'activation':['tanh','tanh'],
#         'lr':0.004409350423921994
#     })}
# )
lstm_hyperparams_dict_tech = dict(
    {'2019': dict({
        'num_layers':2,
        'units':[32,32],
        'optimizer':'Adam',
        'drop_out':[0.1,0.1],
        'activation':['linear','tanh'],
        'lr':0.04130929884537057
    }),
     '2020': dict({
        'num_layers':2,
        'units':[96,32],
        'optimizer':'Adam',
        'drop_out':[0.1,0.1],
        'activation':['tanh','tanh'],
        'lr':0.0017545326289340611
    }),
     '2021': dict({
        'num_layers':2,
        'units':[96,32],
        'optimizer':'Adam',
        'drop_out':[0.1,0.2],
        'activation':['linear','tanh'],
        'lr':0.008514542675036612
    }),
     '2022': dict({
        'num_layers':2,
        'units':[32,32],
        'optimizer':'Adagrad',
        'drop_out':[0.2,0.1],
        'activation':['linear','tanh'],
        'lr':0.06210952184736157
    }),
     '2023': dict({
        'num_layers':4,
        'units':[128,32,32,32],
        'optimizer':'Adam',
        'drop_out':[0.2,0.1,0.1,0.2],
        'activation':['tanh','tanh','linear','linear'],
        'lr':0.019671144721368838
    }),
     '2024': dict({
        'num_layers':2,
        'units':[32,32],
        'optimizer':'Adam',
        'drop_out':[0.2,0.1],
        'activation':['tanh','tanh'],
        'lr':0.004409350423921994
    })}
)

rf_hyperparams_dict_tech = dict(
    {'2019': dict({
        'n_estimators': 300,
        'min_samples_split': 5,
        'min_samples_leaf': 4,
        'max_features': 'log2',
        'max_depth': 20
    }),
     '2020': dict({
        'n_estimators': 300,
        'min_samples_split': 5,
        'min_samples_leaf': 4,
        'max_features': 'log2',
        'max_depth': 20
    }),
     '2021': dict({
        'n_estimators': 300,
        'min_samples_split': 5,
        'min_samples_leaf': 4,
        'max_features': 'log2',
        'max_depth': 20
    }),
     '2022': dict({
        'n_estimators': 100,
        'min_samples_split': 10,
        'min_samples_leaf': 4,
        'max_features': 'log2',
        'max_depth': None
    }),
     '2023': dict({
        'n_estimators': 300,
        'min_samples_split': 5,
        'min_samples_leaf': 4,
        'max_features': 'log2',
        'max_depth': 20
    }),
     '2024': dict({
        'n_estimators': 100,
        'min_samples_split': 5,
        'min_samples_leaf': 4,
        'max_features': 'log2',
        'max_depth': 30
    })}
)

In [10]:
from tensorflow.keras.optimizers import Adam, Adagrad
from tensorflow.keras.models import Sequential, load_model
def overall_function(dataset, outcome,lstm_hyperparams_dict_tech, rf_hyperparams_dict_tech):
    dataset['date'] = pd.to_datetime(dataset['date']) # converting to date format
    dataset = dataset.sort_values(by='date')
    df_factor = dataset.drop(columns=['mkt_return','mth_return','rf','rolling_sharpe', 'rolling_alpha_3f', 'rolling_alpha_4f']) # remove irrelevant variables

    # Creating Lagged and Stepped Datasets
    X_dataset, y_dataset = create_stepped_dataset(create_lagged_dataset(df_factor, lag=1,target_var=outcome, id = 'crsp_fundno'),step=1,target_var=outcome, id = 'crsp_fundno_L1')
    # return(X_dataset)
    X_dataset = X_dataset.drop(columns=['crsp_fundno_L1'], errors='ignore')

    list_of_dates = pd.to_datetime(X_dataset['date_L1'])
    percentile_70 = list_of_dates.quantile(0.7) # 70-30 split
    train_end = list_of_dates.loc[(list_of_dates - percentile_70).abs().idxmin()]
    df_end = list_of_dates.max()
    # return(y_dataset)
    results = []

    ## Implement cross-validation split
    tscv = TimeSeriesSplit(n_splits = 5)
    
    while train_end != df_end:
        
        test_date = generate_next_date(list_of_dates, train_end)
        if pd.isna(test_date):
            break 

        # Process data for modeling
        X_train, X_test, y_train, y_test = process_factor_model(X_dataset, y_dataset, train_end, test_date)
        
        # For Adding Results
        df_in_loop = y_test.copy()
        
        ### Model 1: Lasso Regression
        lasso_cv = LassoCV(cv = tscv, random_state = 18, max_iter = 100000)
        lasso_cv.fit(X_train, y_train)
        
        # Create the Lasso model with the optimal alpha value
        lasso_model = Lasso(alpha = lasso_cv.alpha_)
        lasso_model.fit(X_train, y_train)
        lassopred = lasso_model.predict(X_test)
        # Adding Linear Model
        df_in_loop[f'lasso'] = lassopred
        print('Lasso Done')
        
        ### Model 1: Ridge Regression ###
        ridge_cv = RidgeCV(cv = tscv)
        ridge_cv.fit(X_train, y_train)
    
        ridge_model = Ridge(alpha = ridge_cv.alpha_)
        ridge_model.fit(X_train, y_train)
        
        ridgepred = ridge_model.predict(X_test)
        # Adding Linear Model
        df_in_loop[f'ridge'] = ridgepred
        print('Ridge Done')
        
        ### Model 2: LSTM ###
        X_train_lstm = np.array(X_train).astype(np.float32)
        X_test_lstm = np.array(X_test).astype(np.float32)
        y_train_lstm = np.array(y_train).astype(np.float32)
        
        test_year = test_date.year
        print(f'test_year is {test_year} with hyperparameters of {lstm_hyperparams_dict_tech[str(test_year)]}')
        # lstm_model = load_model(f'best_lstm_model_{train_end.year}.keras')
        
        num_layers = len(lstm_hyperparams_dict_tech[str(test_year)]['units'])
        
        # Building LSTM Model
        lstm_model = Sequential()
        for layer_num in range(num_layers):
            is_last_layer = (layer_num == (num_layers - 1))  # Check if it's the last layer
            lstm_model.add(LSTM(
                units=lstm_hyperparams_dict_tech[str(test_year)]['units'][layer_num], 
                return_sequences=not is_last_layer,  # Only last layer has return_sequences=False
                input_shape=(X_train_lstm.shape[1], 1) if layer_num == 0 else None,  # Define input shape only for the first layer
                activation=lstm_hyperparams_dict_tech[str(test_year)]['activation'][layer_num]
            ))
            lstm_model.add(Dropout(lstm_hyperparams_dict_tech[str(test_year)]['drop_out'][layer_num]))
        # Output Layer
        lstm_model.add(Dense(units=1))
        
        # Compilation
        if lstm_hyperparams_dict_tech[str(test_year)]['optimizer'] == 'Adam':
            lstm_model.compile(
                optimizer=Adam(learning_rate=lstm_hyperparams_dict_tech[str(test_year)]['lr']),
                loss='mean_squared_error',
                metrics=[
                    tf.keras.metrics.RootMeanSquaredError()
                ]
            )
        elif lstm_hyperparams_dict_tech[str(test_year)]['optimizer'] == 'Nadam':
            lstm_model.compile(
                optimizer=Nadam(learning_rate=lstm_hyperparams_dict_tech[str(test_year)]['lr']),
                loss='mean_squared_error',
                metrics=[
                    tf.keras.metrics.RootMeanSquaredError()
                ]
            )
        elif lstm_hyperparams_dict_tech[str(test_year)]['optimizer'] == 'Adagrad':
            lstm_model.compile(
                optimizer=Adagrad(learning_rate=lstm_hyperparams_dict_tech[str(test_year)]['lr']),
                loss='mean_squared_error',
                metrics=[
                    tf.keras.metrics.RootMeanSquaredError()
                ]
            )

        # Early stopping callback
        callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

        # Train the model
        lstm_model.fit(
            X_train_lstm, y_train_lstm,
            epochs=10, batch_size=512,
            callbacks=[callback]
        )

        lstmpred = lstm_model.predict(X_test_lstm)
        df_in_loop[f'lstm'] = lstmpred
        print('LSTM Done')
        
        optimal_param_xgb = xgb_tuner(X_train, y_train, xgb_param_grid)
        xgboost_model = xgb.XGBRegressor(subsample=optimal_param_xgb['subsample'],
                                         n_estimators=optimal_param_xgb['n_estimators'],
                                         max_depth=optimal_param_xgb['max_depth'],
                                         learning_rate=optimal_param_xgb['learning_rate'],
                                         colsample_bytree=optimal_param_xgb['colsample_bytree'],
                                         random_state=18)
        xgboost_model.fit(X_train, y_train)
        df_in_loop[f'xgboost'] = xgboost_model.predict(X_test)
        print('XGBoost done')

        ## 16. LightGBM
        optimal_param_lgbm = lgbm_tuner(X_train, y_train, lgbm_param_grid)
        lightgbm_model = lgb.LGBMRegressor(objective='regression', 
                                           subsample = optimal_param_lgbm['subsample'],
                                           num_leaves = optimal_param_lgbm['num_leaves'],
                                           n_estimators = optimal_param_lgbm['n_estimators'],
                                           max_depth= optimal_param_lgbm['max_depth'],
                                           learning_rate=optimal_param_lgbm['learning_rate'],
                                           colsample_bytree=optimal_param_lgbm['colsample_bytree'],
                                           random_state=18, verbosity=-1)
        lightgbm_model.fit(X_train, y_train)
        df_in_loop[f'lgbm'] = lightgbm_model.predict(X_test)
        print('LGBM done')
                
        ### Model 3: Random Forest Regression ###
        print(f'test_year is {test_year} with hyperparameters of {rf_hyperparams_dict_tech[str(test_year)]}')
        rf_model = RandomForestRegressor(n_estimators=rf_hyperparams_dict_tech[str(test_year)]['n_estimators'], 
                                         min_samples_split=rf_hyperparams_dict_tech[str(test_year)]['min_samples_split'],
                                         min_samples_leaf=rf_hyperparams_dict_tech[str(test_year)]['min_samples_leaf'],
                                         max_features = rf_hyperparams_dict_tech[str(test_year)]['max_features'], 
                                         max_depth=rf_hyperparams_dict_tech[str(test_year)]['max_depth'],
                                         random_state=40, n_jobs=-1)

        rf_model.fit(X_train, y_train.values.ravel())
        rf_pred = rf_model.predict(X_test)
        # return(y_pred)
        df_in_loop[f'rf'] = rf_pred
        
        print('RF Done')
        
        ### Model 5: PCA ###
        X_train = pd.DataFrame(X_train)
        X_test = pd.DataFrame(X_test)

        X_train_pure = X_train
        X_test_pure = X_test
        
        pca = PCA()
        pca.fit(X_train_pure)
        cumulative_variance_ratio = np.cumsum(pca.explained_variance_ratio_)
          #to explain more than 85% of the variance
        num_components = np.where(cumulative_variance_ratio >= 0.85)[0][0] + 1 
        pca_new = PCA(n_components=num_components)
        X_train_pca = pca_new.fit_transform(X_train_pure)
        X_train_pca = pd.DataFrame(X_train_pca)
        X_train_pca.columns = X_train_pca.columns.astype(str)

        X_test_pca = pca_new.transform(X_test_pure)
        X_test_pca = pd.DataFrame(X_test_pca)
        X_test_pca.columns = X_test_pca.columns.astype(str)
        
        pure_factor_model = LinearRegression()
        pure_factor_model.fit(X_train_pca, y_train)
        y_pred = pure_factor_model.predict(X_test_pca)
        df_in_loop[f'pca'] = y_pred
        print('PCA Done')
        
        # Add results into loop
        results.append(df_in_loop)
        train_end = test_date
        num_remaining_dates = len(list(set(date for date in list_of_dates if date > test_date)))
        print(f'{num_remaining_dates} dates remaining')
        
    combined_df = pd.concat(results, ignore_index=True)
    return combined_df

In [11]:
# Run to train the models
y_tech = overall_function(dataset=df_tech, outcome = "rolling_alpha_5f", 
                          lstm_hyperparams_dict_tech=lstm_hyperparams_dict_tech, rf_hyperparams_dict_tech=rf_hyperparams_dict_tech)

C:\Users\wjlwi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\linear_model\_coordinate_descent.py:1656: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Lasso Done
Ridge Done
test_year is 2019 with hyperparameters of {'num_layers': 2, 'units': [32, 32], 'optimizer': 'Adam', 'drop_out': [0.1, 0.1], 'activation': ['linear', 'tanh'], 'lr': 0.04130929884537057}


C:\Users\wjlwi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 7s 34ms/step - loss: 0.0207 - root_mean_squared_error: 0.1307
Epoch 2/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - loss: 0.0010 - root_mean_squared_error: 0.0303    
Epoch 3/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - loss: 0.0035 - root_mean_squared_error: 0.0580
Epoch 4/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - loss: 6.1520e-04 - root_mean_squared_error: 0.0248
Epoch 5/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 3s 35ms/step - loss: 6.1096e-04 - root_mean_squared_error: 0.0247
Epoch 6/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 3s 35ms/step - loss: 5.9978e-04 - root_mean_squared_error: 0.0245
Epoch 7/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - loss: 5.9395e-04 - root_mean_squared_error: 0.0244
Epoch 8/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 3s 35ms/step - loss: 5.6909e-04 - root_mean_squared_error: 0.0238
Epoch 9/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 3s 36ms/step - loss: 6.5562e-04 - root_mean_squared_error: 0.0256
Epoch 10/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 3s 37ms/step - loss: 6.7221e-04 -

C:\Users\wjlwi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


...

Lasso Done
Ridge Done
test_year is 2024 with hyperparameters of {'num_layers': 2, 'units': [32, 32], 'optimizer': 'Adam', 'drop_out': [0.2, 0.1], 'activation': ['tanh', 'tanh'], 'lr': 0.004409350423921994}


C:\Users\wjlwi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 11s 46ms/step - loss: 8.4190e-04 - root_mean_squared_error: 0.0285
Epoch 2/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 5s 46ms/step - loss: 5.2905e-04 - root_mean_squared_error: 0.0230
Epoch 3/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 5s 46ms/step - loss: 5.1069e-04 - root_mean_squared_error: 0.0226
Epoch 4/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 5s 46ms/step - loss: 5.0424e-04 - root_mean_squared_error: 0.0225
Epoch 5/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 5s 45ms/step - loss: 5.0475e-04 - root_mean_squared_error: 0.0225
Epoch 6/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 5s 45ms/step - loss: 4.8658e-04 - root_mean_squared_error: 0.0221
Epoch 7/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 5s 47ms/step - loss: 4.8928e-04 - root_mean_squared_error: 0.0221
Epoch 8/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 5s 45ms/step - loss: 4.8918e-04 - root_mean_squared_error: 0.0221
Epoch 9/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 5s 45ms/step - loss: 4.8923e-04 - root_mean_squared_error: 0.0221
Epoch 10/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 5s 4

C:\Users\wjlwi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LGBM done
test_year is 2024 with hyperparameters of {'n_estimators': 100, 'min_samples_split': 5, 'min_samples_leaf': 4, 'max_features': 'log2', 'max_depth': 30}
RF Done
PCA Done
0 dates remaining


In [12]:
y_tech.to_csv("tech_funds_forecast_tuned.csv")

Random Forest Hyperparameter Tuning (Healthcare)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
tscv = TimeSeriesSplit(n_splits = 5)

df['date'] = pd.to_datetime(df['date']) # converting to date format
df = df.sort_values(by='date')
df_factor = df.drop(columns=['mkt_return','mth_return','rf']) # remove irrelevant variables

# Creating Lagged and Stepped Datasets
X_dataset, y_dataset = create_stepped_dataset(create_lagged_dataset(df_factor, lag=1,target_var='rolling_alpha_5f', id = 'crsp_fundno'),step=1,target_var='rolling_alpha_5f', id = 'crsp_fundno_L1')
# return(X_dataset)
X_dataset = X_dataset.drop(columns=['crsp_fundno_L1'], errors='ignore')

list_of_dates = pd.to_datetime(X_dataset['date_L1'])
percentile_70 = list_of_dates.quantile(0.7) # 70-30 split

train_end = list_of_dates.loc[(list_of_dates - percentile_70).abs().idxmin()]
df_end = list_of_dates.max()

# Define hyperparameter grid
rf_param_grid = {
    'n_estimators': [100, 300, 500, 1000],  # Number of trees
    'max_depth': [10, 20, 30, None],  # Depth of trees
    'min_samples_split': [2, 5, 10],  # Minimum samples for split
    'min_samples_leaf': [1, 2, 4],  # Minimum samples per leaf
    'max_features': ['sqrt', 'log2', None]  # Number of features considered per split
}

def rf_tuner(X_train, y_train, rf_param_grid):
    # Initialize RandomForestRegressor
    rf_model = RandomForestRegressor(random_state=40, n_jobs=-1)

    # Use RandomizedSearchCV for efficiency
    rf_random_search = RandomizedSearchCV(
        estimator=rf_model,
        param_distributions=rf_param_grid,
        n_iter=20,  # Number of parameter settings tested
        cv=tscv,  # TimeSeriesSplit
        scoring='neg_mean_squared_error',
        verbose=2,
        random_state=42,
        n_jobs=-1
    )

    # Perform hyperparameter tuning
    rf_random_search.fit(X_train, y_train.values.ravel())

    # Best hyperparameters
    best_rf_params = rf_random_search.best_params_
    print(f'Best RF Parameters: {best_rf_params}')
    return(best_rf_params)

In [ ]:
hyperparam_lst = []

# 2019
test_date = generate_next_date(list_of_dates, train_end)
X_train, X_test, y_train, y_test = process_factor_model(X_dataset, y_dataset, train_end, test_date)
initial_best_rf_params = rf_tuner(X_train, y_train, rf_param_grid)
hyperparam_lst.append(initial_best_rf_params)
current_year = 2019

while pd.notna(test_date):
    test_date = generate_next_date(list_of_dates, train_end)
    if test_date.year != (current_year + 1): # Still same year: skip and repeat until the year is the next year
        train_end = test_date
        print(f"Train End is {train_end}, year is {train_end.year}, which is same as current year {current_year}. Repeating until following year is obtained")
        continue
    else:
        print(f"Test Date is {test_date}, year is {test_date.year}, which means it is being trained until the end of {current_year}. Conducting Tuning")
        # Process data for modeling
        X_train, X_test, y_train, y_test = process_factor_model(X_dataset, y_dataset, train_end, test_date)
        best_rf_params = rf_tuner(X_train, y_train, rf_param_grid)
        hyperparam_lst.append(best_rf_params)
        train_end = test_date # after tuning, expand the training set
        current_year = current_year + 1

Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best RF Parameters: {'n_estimators': 300, 'min_samples_split': 5, 'min_samples_leaf': 4, 'max_features': 'log2', 'max_depth': 20}
Train End is 2019-12-31 00:00:00, year is 2019, which is same as current year 2019. Repeating until following year is obtained
Test Date is 2020-01-31 00:00:00, year is 2020, which means it is being trained until the end of 2019. Conducting Tuning
Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best RF Parameters: {'n_estimators': 100, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_features': 'log2', 'max_depth': None}
Train End is 2020-02-28 00:00:00, year is 2020, which is same as current year 2020. Repeating until following year is obtained
Train End is 2020-03-31 00:00:00, year is 2020, which is same as current year 2020. Repeating until following year is obtained
Train End is 2020-04-30 00:00:00, year is 2020, which is same as current year 2020. Repeating until following y

LSTM Hyperparam Tuning Functions

In [25]:
from tensorflow.keras.optimizers import Adam, Adagrad, Nadam
def build_lstm_model(hp):
    lstm_model = Sequential()
    # Select the number of LSTM layers (2, 3, or 4)
    num_layers = hp.Choice('num_layers', [2, 3, 4])
    # Choose optimizer
    optimizer_name = hp.Choice('optimizer', ['Adam', 'Adagrad'])
    optimizers = {
        "Adam": Adam(learning_rate=hp.Float('learning_rate', 1e-4, 1e-1, sampling='LOG')),
        "Adagrad": Adagrad(learning_rate=hp.Float('learning_rate', 1e-4, 1e-1, sampling='LOG')),
        "Nadam": Nadam(learning_rate=hp.Float('learning_rate', 1e-4, 1e-1, sampling='LOG'))
    }
    # Layer 1
    first_layer_units = hp.Int('lstm_units_1', min_value=32, max_value=128, step=32)
    lstm_model.add(LSTM(
        units=first_layer_units,
        return_sequences=True,
        input_shape=(X_train_lstm.shape[1], 1),
        activation=hp.Choice(f'activation_1', ['tanh', 'sigmoid', 'linear'])
    ))
    lstm_model.add(Dropout(hp.Float('dropout_1', min_value=0.1, max_value=0.3, step=0.1)))
    
    # Add remaining layers with non-increasing units
    previous_units = first_layer_units  # Track previous layer units
    
    for i in range(1, num_layers):
        available_units = [u for u in [32, 64, 96, 128] if u <= previous_units]  # Only allow non-increasing sizes
        current_units = hp.Choice(f'lstm_units_{i+1}', available_units)

        lstm_model.add(LSTM(
            units=current_units,
            return_sequences=True if i < num_layers - 1 else False,  
            activation=hp.Choice(f'activation_{i+1}', ['tanh', 'sigmoid', 'linear'])
        ))
        lstm_model.add(Dropout(hp.Float(f'dropout_{i+1}', min_value=0.1, max_value=0.3, step=0.1)))
        previous_units = current_units  # Update tracking variable
    
    # Output Layer
    lstm_model.add(Dense(units=1))
    
    # Compile the model
    lstm_model.compile(
        optimizer=optimizers[optimizer_name],
        loss='mean_squared_error',
        metrics=[tf.keras.metrics.RootMeanSquaredError()]
    )
    return lstm_model

def tune_lstm(X_train_lstm, y_train_lstm, year, save_path="best_lstm_model"):
    tuner = kt.BayesianOptimization(
        build_lstm_model,
        objective='val_loss',
        max_trials=50,
        executions_per_trial=1,
        directory=f'lstm_tuning_{year}_tech',
        project_name='fund_forecasting'
    )

    # Perform the search with batch_size as a hyperparameter to tune
    tuner.search(
        X_train_lstm, y_train_lstm,
        epochs=10,
        validation_split=0.2,
        batch_size=512,
        callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)]
    )

    # Get the best hyperparameters
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
    
    # Print the best hyperparameters
    print("Best Hyperparameters:", best_hps.values)

    return best_hps

LSTM Hyperparameter Tuning for 2019 - 2024 --> Yearly Hyperparameter Tuning

In [32]:
df_tech['date'] = pd.to_datetime(df_tech['date']) # converting to date format
df_tech = df_tech.sort_values(by='date')
df_factor = df_tech.drop(columns=['mkt_return','mth_return','rf','rolling_sharpe', 'rolling_alpha_3f', 'rolling_alpha_4f'])

# Creating Lagged and Stepped Datasets
X_dataset, y_dataset = create_stepped_dataset(create_lagged_dataset(df_factor, lag=1,target_var='rolling_alpha_5f', id = 'crsp_fundno'),step=1,target_var='rolling_alpha_5f', id = 'crsp_fundno_L1')
# return(X_dataset)
X_dataset = X_dataset.drop(columns=['crsp_fundno_L1'], errors='ignore')

list_of_dates = pd.to_datetime(X_dataset['date_L1'])
percentile_70 = list_of_dates.quantile(0.7) # 70-30 split

train_end = list_of_dates.loc[(list_of_dates - percentile_70).abs().idxmin()]
df_end = list_of_dates.max()

hyperparam_lst = []

# 2019
current_year = 2019
test_date = generate_next_date(list_of_dates, train_end)
X_train, X_test, y_train, y_test = process_factor_model(X_dataset, y_dataset, train_end, test_date)
X_train_lstm = np.array(X_train).astype(np.float32)
X_test_lstm = np.array(X_test).astype(np.float32)
y_train_lstm = np.array(y_train).astype(np.float32)
best_model_params = tune_lstm(X_train_lstm, y_train_lstm, year = current_year)
hyperparam_lst.append(best_model_params)
# current_year = 2020

while pd.notna(test_date):
    test_date = generate_next_date(list_of_dates, train_end)
    if test_date.year != (current_year + 1): # Still same year: skip and repeat until the year is the next year
        train_end = test_date
        print(f"Train End is {train_end}, year is {train_end.year}, which is same as current year {current_year}. Repeating until following year is obtained")
        continue
    else:
        print(f"Test Date is {test_date}, year is {test_date.year}, which means it is being trained until the end of {current_year}. Conducting Tuning")
        # Process data for modeling
        X_train, X_test, y_train, y_test = process_factor_model(X_dataset, y_dataset, train_end, test_date)
        X_train_lstm = np.array(X_train).astype(np.float32)
        X_test_lstm = np.array(X_test).astype(np.float32)
        y_train_lstm = np.array(y_train).astype(np.float32)
        best_model_params = tune_lstm(X_train_lstm, y_train_lstm, year = current_year+1)
        hyperparam_lst.append(best_model_params)
        train_end = test_date # after tuning, expand the training set
        current_year = current_year + 1

Reloading Tuner from lstm_tuning_2019_tech\fund_forecasting\tuner0.json
Best Hyperparameters: {'num_layers': 2, 'optimizer': 'Adam', 'learning_rate': 0.04130929884537057, 'lstm_units_1': 32, 'activation_1': 'linear', 'dropout_1': 0.1, 'lstm_units_2': 32, 'activation_2': 'tanh', 'dropout_2': 0.1, 'lstm_units_3': 32, 'activation_3': 'tanh', 'dropout_3': 0.1, 'lstm_units_4': 32, 'activation_4': 'sigmoid', 'dropout_4': 0.2}
Train End is 2019-02-28 00:00:00, year is 2019, which is same as current year 2019. Repeating until following year is obtained
Train End is 2019-03-29 00:00:00, year is 2019, which is same as current year 2019. Repeating until following year is obtained
Train End is 2019-04-30 00:00:00, year is 2019, which is same as current year 2019. Repeating until following year is obtained
Train End is 2019-05-31 00:00:00, year is 2019, which is same as current year 2019. Repeating until following year is obtained
Train End is 2019-06-28 00:00:00, year is 2019, which is same as cur

In [29]:
from sklearn.model_selection import RandomizedSearchCV
tscv = TimeSeriesSplit(n_splits = 5)

df_tech['date'] = pd.to_datetime(df_tech['date']) # converting to date format
df_tech = df_tech.sort_values(by='date')
df_factor = df_tech.drop(columns=['mkt_return','mth_return','rf','rolling_sharpe', 'rolling_alpha_3f', 'rolling_alpha_4f'])

# Creating Lagged and Stepped Datasets
X_dataset, y_dataset = create_stepped_dataset(create_lagged_dataset(df_factor, lag=1,target_var='rolling_alpha_5f', id = 'crsp_fundno'),step=1,target_var='rolling_alpha_5f', id = 'crsp_fundno_L1')
# return(X_dataset)
X_dataset = X_dataset.drop(columns=['crsp_fundno_L1'], errors='ignore')

list_of_dates = pd.to_datetime(X_dataset['date_L1'])
percentile_70 = list_of_dates.quantile(0.7) # 70-30 split

train_end = list_of_dates.loc[(list_of_dates - percentile_70).abs().idxmin()]
df_end = list_of_dates.max()

# Define hyperparameter grid
rf_param_grid = {
    'n_estimators': [100, 300, 500, 1000],  # Number of trees
    'max_depth': [10, 20, 30, None],  # Depth of trees
    'min_samples_split': [2, 5, 10],  # Minimum samples for split
    'min_samples_leaf': [1, 2, 4],  # Minimum samples per leaf
    'max_features': ['sqrt', 'log2', None]  # Number of features considered per split
}

def rf_tuner(X_train, y_train, rf_param_grid):
    # Initialize RandomForestRegressor
    rf_model = RandomForestRegressor(random_state=40, n_jobs=-1)

    # Use RandomizedSearchCV for efficiency
    rf_random_search = RandomizedSearchCV(
        estimator=rf_model,
        param_distributions=rf_param_grid,
        n_iter=20,  # Number of parameter settings tested
        cv=tscv,  # TimeSeriesSplit
        scoring='neg_mean_squared_error',
        verbose=2,
        random_state=42,
        n_jobs=-1
    )

    # Perform hyperparameter tuning
    rf_random_search.fit(X_train, y_train.values.ravel())

    # Best hyperparameters
    best_rf_params = rf_random_search.best_params_
    print(f'Best RF Parameters: {best_rf_params}')
    return(best_rf_params)

In [30]:
hyperparam_lst_rf = []

# 2019
test_date = generate_next_date(list_of_dates, train_end)
X_train, X_test, y_train, y_test = process_factor_model(X_dataset, y_dataset, train_end, test_date)
initial_best_rf_params = rf_tuner(X_train, y_train, rf_param_grid)
hyperparam_lst_rf.append(initial_best_rf_params)
current_year = 2019

while pd.notna(test_date):
    test_date = generate_next_date(list_of_dates, train_end)
    if test_date.year != (current_year + 1): # Still same year: skip and repeat until the year is the next year
        train_end = test_date
        print(f"Train End is {train_end}, year is {train_end.year}, which is same as current year {current_year}. Repeating until following year is obtained")
        continue
    else:
        print(f"Test Date is {test_date}, year is {test_date.year}, which means it is being trained until the end of {current_year}. Conducting Tuning")
        # Process data for modeling
        X_train, X_test, y_train, y_test = process_factor_model(X_dataset, y_dataset, train_end, test_date)
        best_rf_params = rf_tuner(X_train, y_train, rf_param_grid)
        hyperparam_lst_rf.append(best_rf_params)
        train_end = test_date # after tuning, expand the training set
        current_year = current_year + 1

Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best RF Parameters: {'n_estimators': 300, 'min_samples_split': 5, 'min_samples_leaf': 4, 'max_features': 'log2', 'max_depth': 20}
Train End is 2019-02-28 00:00:00, year is 2019, which is same as current year 2019. Repeating until following year is obtained
Train End is 2019-03-29 00:00:00, year is 2019, which is same as current year 2019. Repeating until following year is obtained
Train End is 2019-04-30 00:00:00, year is 2019, which is same as current year 2019. Repeating until following year is obtained
Train End is 2019-05-31 00:00:00, year is 2019, which is same as current year 2019. Repeating until following year is obtained
Train End is 2019-06-28 00:00:00, year is 2019, which is same as current year 2019. Repeating until following year is obtained
Train End is 2019-07-31 00:00:00, year is 2019, which is same as current year 2019. Repeating until following year is obtained
Train End is 2019-08-30 00:00:00, year is 2019

In [31]:
hyperparam_lst_rf

[{'n_estimators': 300,
  'min_samples_split': 5,
  'min_samples_leaf': 4,
  'max_features': 'log2',
  'max_depth': 20},
 {'n_estimators': 300,
  'min_samples_split': 5,
  'min_samples_leaf': 4,
  'max_features': 'log2',
  'max_depth': 20},
 {'n_estimators': 300,
  'min_samples_split': 5,
  'min_samples_leaf': 4,
  'max_features': 'log2',
  'max_depth': 20},
 {'n_estimators': 100,
  'min_samples_split': 10,
  'min_samples_leaf': 4,
  'max_features': 'log2',
  'max_depth': None},
 {'n_estimators': 300,
  'min_samples_split': 5,
  'min_samples_leaf': 4,
  'max_features': 'log2',
  'max_depth': 20},
 {'n_estimators': 100,
  'min_samples_split': 5,
  'min_samples_leaf': 4,
  'max_features': 'log2',
  'max_depth': 30}]

In [ ]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam, Adagrad, Nadam
import keras_tuner as kt
tf.compat.v1.enable_eager_execution()
# Convert date column to datetime and sort
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(by='date')

# Remove irrelevant variables
df_factor = df.drop(columns=['mkt_return', 'mth_return', 'rf'], errors='ignore')

# Create lagged and stepped datasets
X_dataset, y_dataset = create_stepped_dataset(
    create_lagged_dataset(df_factor, lag=1, target_var='rolling_alpha_5f', id='crsp_fundno'),
    step=1,
    target_var='rolling_alpha_5f',
    id='crsp_fundno_L1'
)

# Drop fund ID column (if present)
X_dataset = X_dataset.drop(columns=['crsp_fundno_L1'], errors='ignore')

# Define train-test split based on date
list_of_dates = pd.to_datetime(X_dataset['date_L1'])
percentile_70 = list_of_dates.quantile(0.7)

train_end = list_of_dates.loc[(list_of_dates - percentile_70).abs().idxmin()]
df_end = list_of_dates.max()

hyperparam_lst = []

# Start from 2019
current_year = 2018
test_date = generate_next_date(list_of_dates, train_end)
pretrained_model = None

# ---- Function to Build LSTM Model ----
def build_lstm_model(hp):
    lstm_model = Sequential()
    
    num_layers = hp.Choice('num_layers', [2, 3, 4])  # Choose number of LSTM layers
    optimizer_name = hp.Choice('optimizer', ['Adam', 'Adagrad', 'Nadam'])  # Choose optimizer

    optimizers = {
        "Adam": Adam(learning_rate=hp.Float('learning_rate', 1e-4, 1e-1, sampling='LOG')),
        "Adagrad": Adagrad(learning_rate=hp.Float('learning_rate', 1e-4, 1e-1, sampling='LOG')),
        "Nadam": Nadam(learning_rate=hp.Float('learning_rate', 1e-4, 1e-1, sampling='LOG'))
    }
    
    first_layer_units = hp.Int('lstm_units_1', min_value=32, max_value=128, step=32)
    lstm_model.add(LSTM(
        units=first_layer_units,
        return_sequences=True,
        input_shape=(X_train_lstm.shape[1], 1),
        activation=hp.Choice('activation_1', ['tanh', 'sigmoid', 'linear'])
    ))
    lstm_model.add(Dropout(hp.Float('dropout_1', min_value=0.1, max_value=0.3, step=0.1)))

    previous_units = first_layer_units

    for i in range(1, num_layers):
        available_units = [u for u in [32, 64, 96, 128] if u <= previous_units]
        current_units = hp.Choice(f'lstm_units_{i+1}', available_units)

        lstm_model.add(LSTM(
            units=current_units,
            return_sequences=True if i < num_layers - 1 else False,
            activation=hp.Choice(f'activation_{i+1}', ['tanh', 'sigmoid', 'linear'])
        ))
        lstm_model.add(Dropout(hp.Float(f'dropout_{i+1}', min_value=0.1, max_value=0.3, step=0.1)))
        previous_units = current_units  

    # Output layer
    lstm_model.add(Dense(units=1))

    # Compile model
    lstm_model.compile(
        optimizer=optimizers[optimizer_name],
        loss='mean_squared_error',
        metrics=[tf.keras.metrics.RootMeanSquaredError()]
    )
    return lstm_model

# ---- Function to Tune or Fine-Tune LSTM ----
def tune_lstm(X_train_lstm, y_train_lstm, year, pretrained_model=None, save_path="best_lstm_model"):
    """Trains or fine-tunes an LSTM model with hyperparameter tuning."""
    
    model_path = f"{save_path}_{year}.keras"
    
    if pretrained_model and os.path.exists(pretrained_model):
        print(f"Loading pretrained model from {pretrained_model}...")
        lstm_model = load_model(pretrained_model)
    else:
        print("No pretrained model found. Running full hyperparameter tuning...")
        tuner = kt.BayesianOptimization(
            build_lstm_model,
            objective='val_loss',
            max_trials=50,
            executions_per_trial=1,
            directory=f'lstm_tuning_{year}',
            project_name='fund_forecasting'
        )

        tuner.search(
            X_train_lstm, y_train_lstm,
            epochs=50,
            validation_split=0.2,
            batch_size=128,
            callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)]
        )

        best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
        print(f"Best Hyperparameters for {year}: {best_hps.values}")
        lstm_model = tuner.hypermodel.build(best_hps)

    # Train (or fine-tune) the model
    lstm_model.fit(
        X_train_lstm, y_train_lstm,
        epochs=20,
        validation_split=0.2,
        batch_size=128,
        callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)]
    )

    # Save model for next year's training
    lstm_model.save(model_path)
    print(f"Model saved as {model_path}")
    
    return model_path  # Return saved model path

# ---- Loop Through Years for Transfer Learning ----
while pd.notna(test_date):
    test_date = generate_next_date(list_of_dates, train_end)
    
    if test_date.year != (current_year + 1):
        train_end = test_date
        print(f"Train End: {train_end}, same year {current_year}. Repeating until next year.")
        continue

    print(f"\nTraining {current_year} model until {test_date}. Conducting hyperparameter tuning...")
    
    # Prepare data for training
    X_train, X_test, y_train, y_test = process_factor_model(X_dataset, y_dataset, train_end, test_date)
    X_train_lstm = np.array(X_train).astype(np.float32)
    X_test_lstm = np.array(X_test).astype(np.float32)
    y_train_lstm = np.array(y_train).astype(np.float32)

    # Train or fine-tune model
    pretrained_model = tune_lstm(X_train_lstm, y_train_lstm, year=current_year+1, pretrained_model=pretrained_model)
    hyperparam_lst.append(pretrained_model)

    # Update year and training end date
    train_end = test_date
    current_year += 1


Training 2018 model until 2019-12-31 00:00:00. Conducting hyperparameter tuning...
No pretrained model found. Running full hyperparameter tuning...
Reloading Tuner from lstm_tuning_2019\fund_forecasting\tuner0.json
Best Hyperparameters for 2019: {'num_layers': 4, 'optimizer': 'Nadam', 'learning_rate': 0.007390757774868635, 'lstm_units_1': 32, 'activation_1': 'linear', 'dropout_1': 0.1, 'lstm_units_2': 32, 'activation_2': 'tanh', 'dropout_2': 0.2, 'lstm_units_3': 32, 'activation_3': 'sigmoid', 'dropout_3': 0.1, 'lstm_units_4': 32, 'activation_4': 'tanh', 'dropout_4': 0.1}


C:\Users\wjlwi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 34s 116ms/step - loss: 0.0245 - root_mean_squared_error: 0.1340 - val_loss: 2.4114e-04 - val_root_mean_squared_error: 0.0155
Epoch 2/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 21s 106ms/step - loss: 5.5111e-04 - root_mean_squared_error: 0.0235 - val_loss: 2.4324e-04 - val_root_mean_squared_error: 0.0156
Epoch 3/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 21s 104ms/step - loss: 5.2553e-04 - root_mean_squared_error: 0.0229 - val_loss: 2.5315e-04 - val_root_mean_squared_error: 0.0159
Epoch 4/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 21s 104ms/step - loss: 4.9586e-04 - root_mean_squared_error: 0.0223 - val_loss: 2.3901e-04 - val_root_mean_squared_error: 0.0155
Epoch 5/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 21s 106ms/step - loss: 4.8268e-04 - root_mean_squared_error: 0.0220 - val_loss: 2.5097e-04 - val_root_mean_squared_error: 0.0158
Epoch 6/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 22s 108ms/step - loss: 5.0658e-04 - root_mean_squared_error: 0.0225 - val_loss: 2.2655e-04 - val_root_mean_squared_erro

In [ ]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam, Adagrad, Nadam
import keras_tuner as kt
tf.compat.v1.enable_eager_execution()
# Convert date column to datetime and sort
df_tech['date'] = pd.to_datetime(df_tech['date'])
df_tech = df_tech.sort_values(by='date')

# Remove irrelevant variables
df_factor = df_tech.drop(columns=['mkt_return', 'mth_return', 'rf'], errors='ignore')

# Create lagged and stepped datasets
X_dataset, y_dataset = create_stepped_dataset(
    create_lagged_dataset(df_factor, lag=1, target_var='rolling_alpha_5f', id='crsp_fundno'),
    step=1,
    target_var='rolling_alpha_5f',
    id='crsp_fundno_L1'
)

# Drop fund ID column (if present)
X_dataset = X_dataset.drop(columns=['crsp_fundno_L1'], errors='ignore')

# Define train-test split based on date
list_of_dates = pd.to_datetime(X_dataset['date_L1'])
percentile_70 = list_of_dates.quantile(0.7)

train_end = list_of_dates.loc[(list_of_dates - percentile_70).abs().idxmin()]
df_end = list_of_dates.max()

hyperparam_lst = []

# Start from 2019
current_year = 2018
test_date = generate_next_date(list_of_dates, train_end)
pretrained_model = None

# ---- Function to Build LSTM Model ----
def build_lstm_model(hp):
    lstm_model = Sequential()
    
    num_layers = hp.Choice('num_layers', [2, 3, 4])  # Choose number of LSTM layers
    optimizer_name = hp.Choice('optimizer', ['Adam', 'Adagrad', 'Nadam'])  # Choose optimizer

    optimizers = {
        "Adam": Adam(learning_rate=hp.Float('learning_rate', 1e-4, 1e-1, sampling='LOG')),
        "Adagrad": Adagrad(learning_rate=hp.Float('learning_rate', 1e-4, 1e-1, sampling='LOG')),
        "Nadam": Nadam(learning_rate=hp.Float('learning_rate', 1e-4, 1e-1, sampling='LOG'))
    }
    
    first_layer_units = hp.Int('lstm_units_1', min_value=32, max_value=128, step=32)
    lstm_model.add(LSTM(
        units=first_layer_units,
        return_sequences=True,
        input_shape=(X_train_lstm.shape[1], 1),
        activation=hp.Choice('activation_1', ['tanh', 'sigmoid', 'linear'])
    ))
    lstm_model.add(Dropout(hp.Float('dropout_1', min_value=0.1, max_value=0.3, step=0.1)))

    previous_units = first_layer_units

    for i in range(1, num_layers):
        available_units = [u for u in [32, 64, 96, 128] if u <= previous_units]
        current_units = hp.Choice(f'lstm_units_{i+1}', available_units)

        lstm_model.add(LSTM(
            units=current_units,
            return_sequences=True if i < num_layers - 1 else False,
            activation=hp.Choice(f'activation_{i+1}', ['tanh', 'sigmoid', 'linear'])
        ))
        lstm_model.add(Dropout(hp.Float(f'dropout_{i+1}', min_value=0.1, max_value=0.3, step=0.1)))
        previous_units = current_units  

    # Output layer
    lstm_model.add(Dense(units=1))

    # Compile model
    lstm_model.compile(
        optimizer=optimizers[optimizer_name],
        loss='mean_squared_error',
        metrics=[tf.keras.metrics.RootMeanSquaredError()]
    )
    return lstm_model

# ---- Function to Tune or Fine-Tune LSTM ----
def tune_lstm(X_train_lstm, y_train_lstm, year, pretrained_model=None, save_path="best_lstm_model"):
    """Trains or fine-tunes an LSTM model with hyperparameter tuning."""
    
    model_path = f"{save_path}_{year}_tech.keras"
    
    if pretrained_model and os.path.exists(pretrained_model):
        print(f"Loading pretrained model from {pretrained_model}...")
        lstm_model = load_model(pretrained_model)
    else:
        print("No pretrained model found. Running full hyperparameter tuning...")
        tuner = kt.BayesianOptimization(
            build_lstm_model,
            objective='val_loss',
            max_trials=50,
            executions_per_trial=1,
            directory=f'lstm_tuning_{year}',
            project_name='fund_forecasting'
        )

        tuner.search(
            X_train_lstm, y_train_lstm,
            epochs=50,
            validation_split=0.2,
            batch_size=128,
            callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)]
        )

        best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
        print(f"Best Hyperparameters for {year}: {best_hps.values}")
        lstm_model = tuner.hypermodel.build(best_hps)

    # Train (or fine-tune) the model
    lstm_model.fit(
        X_train_lstm, y_train_lstm,
        epochs=20,
        validation_split=0.2,
        batch_size=128,
        callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)]
    )

    # Save model for next year's training
    lstm_model.save(model_path)
    print(f"Model saved as {model_path}")
    
    return model_path  # Return saved model path

# ---- Loop Through Years for Transfer Learning ----
while pd.notna(test_date):
    test_date = generate_next_date(list_of_dates, train_end)
    
    if test_date.year != (current_year + 1):
        train_end = test_date
        print(f"Train End: {train_end}, same year {current_year}. Repeating until next year.")
        continue

    print(f"\nTraining {current_year} model until {test_date}. Conducting hyperparameter tuning...")
    
    # Prepare data for training
    X_train, X_test, y_train, y_test = process_factor_model(X_dataset, y_dataset, train_end, test_date)
    X_train_lstm = np.array(X_train).astype(np.float32)
    X_test_lstm = np.array(X_test).astype(np.float32)
    y_train_lstm = np.array(y_train).astype(np.float32)

    # Train or fine-tune model
    pretrained_model = tune_lstm(X_train_lstm, y_train_lstm, year=current_year+1, pretrained_model=pretrained_model)
    hyperparam_lst.append(pretrained_model)

    # Update year and training end date
    train_end = test_date
    current_year += 1

In [ ]:
# lstm_hyperparams_dict_healthcare = dict(
#     {'2019': dict({
#         'num_layers':2,
#         'units':[32,32],
#         'optimizer':'Adam',
#         'drop_out':[0.2,0.2],
#         'activation':['tanh','sigmoid'],
#         'lr':0.01995625160697196
#     }),
#      '2020': dict({
#         'num_layers':3,
#         'units':[32,32,32],
#         'optimizer':'Adam',
#         'drop_out':[0.2,0.2,0.1],
#         'activation':['linear','sigmoid','sigmoid'],
#         'lr':0.0011839804874350056
#     }),
#      '2021': dict({
#         'num_layers':2,
#         'units':[128,32],
#         'optimizer':'Adam',
#         'drop_out':[0.2,0.1],
#         'activation':['tanh','linear'],
#         'lr':0.018389663547277172
#     }),
#      '2022': dict({
#         'num_layers':4,
#         'units':[64,32,32,32],
#         'optimizer':'Adam',
#         'drop_out':[0.1,0.1,0.2,0.2],
#         'activation':['linear','tanh','tanh','linear'],
#         'lr':0.011965744369339311
#     }),
#      '2023': dict({
#         'num_layers':2,
#         'units':[32,32],
#         'optimizer':'Adam',
#         'drop_out':[0.2,0.1],
#         'activation':['tanh','tanh'],
#         'lr':0.0005130193097484537
#     }),
#      '2024': dict({
#         'num_layers':2,
#         'units':[32,32],
#         'optimizer':'Adam',
#         'drop_out':[0.1,0.2],
#         'activation':['linear','tanh'],
#         'lr':0.0035163892485339547
#     })}
# )